# 🤖 Entrenamiento del Modelo
## FlightOnTime - Predicción de Retrasos de Vuelos

---

### Objetivos:
1. Cargar datos preprocesados
2. Aplicar feature engineering
3. Entrenar modelos (Logistic Regression y Random Forest)
4. Evaluar y comparar modelos
5. Seleccionar el mejor modelo
6. Guardar modelo y metadatos
7. Realizar ejemplo de predicción

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ Librerías importadas")

## 1️⃣ Cargar y Preprocesar Datos

In [ ]:
# Importar módulos del proyecto
import sys
sys.path.append('..')  # Para importar desde src/

from src.config import (
    get_raw_data_path,
    TARGET_COLUMN,
    DELAY_THRESHOLD,
    RANDOM_STATE,
    CATEGORICAL_FEATURES,
    NUMERIC_FEATURES
)
from src.preprocessing import preprocess_data
from src.features import engineer_features, select_features_for_modeling
from src.modeling import (
    create_preprocessing_pipeline,
    split_train_test,
    train_logistic_regression,
    train_random_forest,
    save_model,
    create_model_metadata
)
from src.evaluation import evaluate_model, compare_models

print("✓ Módulos del proyecto importados")

In [ ]:
# Cargar y preprocesar datos
raw_path = get_raw_data_path()
print(f"📂 Cargando datos desde: {raw_path}\n")

df = preprocess_data(raw_path, sample_size=None)

## 2️⃣ Feature Engineering

In [ ]:
# Buscar columna de fecha
date_col = None
for col in ['fl_date', 'date', 'flight_date']:
    if col in df.columns:
        date_col = col
        break

# Aplicar feature engineering
if date_col:
    df = engineer_features(df, date_column=date_col, create_interactions=False)
else:
    print("⚠️ No se encontró columna de fecha. Continuando sin features temporales...")

print(f"\n✓ Dataset con features: {df.shape}")

In [ ]:
# Seleccionar features para modelado
X, y, cat_features, num_features = select_features_for_modeling(df, TARGET_COLUMN)

print(f"\n📊 Resumen de features:")
print(f"  Categóricas ({len(cat_features)}): {cat_features}")
print(f"  Numéricas ({len(num_features)}): {num_features}")
print(f"  Total: {len(X.columns)} features")
print(f"  Registros: {len(X):,}")

## 3️⃣ División Train/Test

In [ ]:
# Split estratificado para mantener balance de clases
X_train, X_test, y_train, y_test = split_train_test(
    X, y,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=True
)

## 4️⃣ Crear Pipeline de Preprocesamiento

In [ ]:
# Pipeline con ColumnTransformer
# - Categóricas: Imputación + OneHotEncoding
# - Numéricas: Imputación con mediana

preprocessor = create_preprocessing_pipeline(cat_features, num_features)

## 5️⃣ Entrenar Modelos

### 5.1 Logistic Regression (Baseline)

In [ ]:
# Entrenar Logistic Regression como modelo baseline
model_lr = train_logistic_regression(X_train, y_train, preprocessor)

In [ ]:
# Evaluar Logistic Regression
metrics_lr = evaluate_model(
    model_lr,
    X_test,
    y_test,
    model_name="Logistic Regression",
    save_outputs=True
)

### 5.2 Random Forest

In [ ]:
# Entrenar Random Forest
model_rf = train_random_forest(X_train, y_train, preprocessor)

In [ ]:
# Evaluar Random Forest
metrics_rf = evaluate_model(
    model_rf,
    X_test,
    y_test,
    model_name="Random Forest",
    save_outputs=True
)

## 6️⃣ Comparar Modelos

In [ ]:
# Comparar por métrica principal (Recall de clase Retrasado)
models_metrics = {
    'Logistic_Regression': metrics_lr,
    'Random_Forest': metrics_rf
}

best_model_name = compare_models(models_metrics, primary_metric='recall')

## 7️⃣ Seleccionar y Guardar Mejor Modelo

In [ ]:
# Seleccionar mejor modelo
if best_model_name == 'Logistic_Regression':
    best_model = model_lr
    best_metrics = metrics_lr
else:
    best_model = model_rf
    best_metrics = metrics_rf

print(f"\n🏆 Modelo seleccionado: {best_model_name}")

In [ ]:
# Crear metadatos
metadata = create_model_metadata(
    model_name=best_model_name,
    categorical_features=cat_features,
    numeric_features=num_features,
    metrics=best_metrics
)

print("\n📄 Metadatos del modelo:")
import json
print(json.dumps(metadata, indent=2))

In [ ]:
# Guardar modelo y metadatos
from src.config import MODEL_PATH, METADATA_PATH

save_model(
    model=best_model,
    model_path=MODEL_PATH,
    metadata_path=METADATA_PATH,
    metadata=metadata
)

print("\n✅ Modelo guardado exitosamente")

## 8️⃣ Ejemplo de Predicción

### Predecir un vuelo ficticio

In [ ]:
# Crear vuelo de ejemplo
# Ajusta los valores según las columnas disponibles en tu dataset

vuelo_ejemplo = pd.DataFrame({
    # Características numéricas
    'month': [7],          # Julio
    'day_of_week': [4],    # Viernes
    'hour': [17],          # 5 PM
    'is_weekend': [0],     # No es fin de semana
})

# Agregar columnas categóricas si existen en tu dataset
if 'airline' in cat_features:
    vuelo_ejemplo['airline'] = ['American Airlines']
if 'origin' in cat_features:
    vuelo_ejemplo['origin'] = ['JFK']
if 'dest' in cat_features:
    vuelo_ejemplo['dest'] = ['LAX']
if 'time_slot' in cat_features:
    vuelo_ejemplo['time_slot'] = ['tarde']

print("✈️ Vuelo de ejemplo:")
print(vuelo_ejemplo)

In [ ]:
# Hacer predicción
prediccion = best_model.predict(vuelo_ejemplo)[0]
probabilidades = best_model.predict_proba(vuelo_ejemplo)[0]

# Interpretar resultados
resultado = "Retrasado" if prediccion == 1 else "Puntual"
prob_puntual = probabilidades[0]
prob_retrasado = probabilidades[1]

print("\n" + "="*60)
print("🔮 PREDICCIÓN DEL MODELO")
print("="*60)
print(f"\n  Clase predicha: {prediccion}")
print(f"  Resultado: {resultado}")
print(f"\n  Probabilidades:")
print(f"    - Puntual (0): {prob_puntual:.2%}")
print(f"    - Retrasado (1): {prob_retrasado:.2%}")
print("\n" + "="*60)

# Interpretación
if prediccion == 1:
    print(f"⚠️  Se predice que el vuelo tendrá un retraso > {DELAY_THRESHOLD} minutos")
    print(f"   Confianza: {prob_retrasado:.1%}")
else:
    print(f"✅ Se predice que el vuelo será puntual (retraso ≤ {DELAY_THRESHOLD} minutos)")
    print(f"   Confianza: {prob_puntual:.1%}")

## 9️⃣ Función Reutilizable para Predicciones

In [ ]:
def predecir_retraso(vuelo_data: dict) -> dict:
    """
    Predice si un vuelo será retrasado.
    
    Args:
        vuelo_data: Diccionario con características del vuelo
    
    Returns:
        Diccionario con predicción y probabilidades
    """
    import joblib
    from src.config import MODEL_PATH
    
    # Cargar modelo
    model = joblib.load(MODEL_PATH)
    
    # Convertir a DataFrame
    df_vuelo = pd.DataFrame([vuelo_data])
    
    # Predecir
    prediccion = model.predict(df_vuelo)[0]
    probabilidades = model.predict_proba(df_vuelo)[0]
    
    resultado = {
        'prediccion': int(prediccion),
        'clase': 'Retrasado' if prediccion == 1 else 'Puntual',
        'probabilidad_puntual': float(probabilidades[0]),
        'probabilidad_retrasado': float(probabilidades[1]),
        'confianza': float(max(probabilidades))
    }
    
    return resultado


# Ejemplo de uso
vuelo_prueba = {
    'month': 12,
    'day_of_week': 5,  # Sábado
    'hour': 20,
    'is_weekend': 1,
    'airline': 'Delta',
    'origin': 'ATL',
    'dest': 'ORD',
    'time_slot': 'noche'
}

resultado = predecir_retraso(vuelo_prueba)

print("\n🔮 Resultado de la predicción:")
print(f"  Clase: {resultado['clase']}")
print(f"  Probabilidad de retraso: {resultado['probabilidad_retrasado']:.2%}")
print(f"  Confianza: {resultado['confianza']:.2%}")

## 🎯 Resumen Final

### ✅ Tareas Completadas:

1. ✓ Carga y preprocesamiento de datos
2. ✓ Feature engineering (features temporales y categóricas)
3. ✓ División train/test estratificada
4. ✓ Entrenamiento de modelos:
   - Logistic Regression (baseline)
   - Random Forest
5. ✓ Evaluación con múltiples métricas:
   - Accuracy, Precision, Recall, F1-Score
   - ROC-AUC y Average Precision
   - Matriz de confusión
   - Curvas ROC y Precision-Recall
6. ✓ Selección del mejor modelo (prioridad: Recall)
7. ✓ Guardado de modelo y metadatos
8. ✓ Ejemplo de predicción en producción

### 📊 Archivos Generados:

- `models/model.joblib` - Pipeline completo del modelo
- `models/metadata.json` - Metadatos y métricas
- `outputs/metrics/` - Reportes de evaluación
- `outputs/figures/` - Gráficas de evaluación

### 🚀 Próximos Pasos (Opcional):

1. **Optimización de hiperparámetros**: GridSearchCV o RandomizedSearchCV
2. **Threshold tuning**: Ajustar umbral de decisión según costo de errores
3. **Interpretabilidad**: SHAP values para explicar predicciones
4. **Despliegue**: API REST con FastAPI o Flask
5. **Monitoreo**: Tracking de drift en producción

---

**Proyecto FlightOnTime completado** ✅🎉  
¡Listo para el hackathon!